In [ ]:
#1-1

import os
import pandas as pd
from collections import defaultdict

# Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 이미지가 저장된 폴더 경로
image_folder_path = '/content/drive/MyDrive/2024_dataset/training_image'  # 실제 드라이브 경로로 변경 필요

# 분류 결과 저장을 위한 딕셔너리
gender_count = defaultdict(lambda: defaultdict(int))

# 이미지 파일 탐색
for filename in os.listdir(image_folder_path):
    # 확장자가 모두 .jpg라 필터링 필요 없음
    # 파일 이름을 언더스코어('_')로 분리
    parts = filename.split('_')

    # 성별 추출 (파일명 끝의 M 또는 F)
    gender_code = parts[-1][0]  # M 또는 F 추출
    if gender_code == 'M':
        gender = 'man'
    elif gender_code == 'W':
        gender = 'woman'
    else:
        continue

    # 스타일 추출 (세 번째 요소, 예: 'sportivecasual')
    style = parts[3].split('.')[0]  # 확장자 제거

    # 카운트 증가
    gender_count[gender][style] += 1



# 결과를 pandas 데이터프레임으로 변환하여 보기 좋게 출력
df = pd.DataFrame(gender_count)

df = df.fillna(0)

print(df)

# 결과를 파일로 저장하고 싶다면 다음을 사용할 수 있습니다.
#df.to_csv('/content/drive/MyDrive/gender_style_count.csv', index=True)


In [12]:
model = models.resnet18(pretrained=False)  # Pretrained weights 사용 안 함
model.fc = nn.Linear(model.fc.in_features, num_classes)  # 마지막 레이어를 클래스 수에 맞게 조정

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Custom Dataset Class 정의
class GenderStyleDataset(Dataset):
    def __init__(self, image_folder, label_mapping, transform=None):
        self.image_folder = image_folder
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
        self.transform = transform
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path)

        # 파일명에서 성별과 스타일 추출
        parts = img_name.split('_')
        style = parts[3]  # 예: sportivecasual
        gender_code = parts[-1][0]  # M or W

        if gender_code == 'M':
            gender = 'man'
        elif gender_code == 'W':
            gender = 'woman'

        # 성별 & 스타일을 결합한 클래스 라벨
        class_label = f"{gender}_{style}"  # 예: man_sportivecasual

        # Transform 적용
        if self.transform:
            image = self.transform(image)

        # 문자열 라벨을 숫자 라벨로 변환
        label = self.label_mapping[class_label]

        return image, label

# 라벨 인덱스 매핑 생성
def create_label_mapping(image_folder):
    label_set = set()
    for filename in os.listdir(image_folder):
        parts = filename.split('_')
        style = parts[3]
        gender_code = parts[-1][0]
        if gender_code == 'M':
            gender = 'man'
        elif gender_code == 'W':
            gender = 'woman'
        class_label = f"{gender}_{style}"
        label_set.add(class_label)
    label_mapping = {label: idx for idx, label in enumerate(sorted(label_set))}
    return label_mapping

# 데이터 경로 및 변환 설정
image_folder_path = '/content/drive/MyDrive/2024_dataset/training_image'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 라벨 매핑 생성
label_mapping = create_label_mapping(image_folder_path)
num_classes = len(label_mapping)  # 클래스 수 설정

# 데이터셋 및 DataLoader 생성
dataset = GenderStyleDataset(image_folder=image_folder_path, label_mapping=label_mapping, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# ResNet-18 모델 정의 (pretrained=False로 파라미터 무작위 초기화)
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # 마지막 레이어 수정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# 검증 단계 (추가 가능)
